# Pipeline: spaCy → chunking → Ollama walidacja hintów → placeholdery
Przykład end-to-end: surowy tekst → preprocess (spaCy) → jeden chunk na hint → klasyfikacja w Ollama → ewentualna zamiana na placeholdery.


## 0. (opcjonalnie) instalacja
Zakładamy, że środowisko ma `spacy`, model `pl_core_news_md` oraz `ollama`. W razie potrzeby:
```bash
pip install -r requirements.txt
python -m spacy download pl_core_news_md
```


In [1]:
from spacy_processing import SpacyPreprocessor
from context_manager import ContextManager
from ollama_classifier import OllamaEntityClassifier
from classes import EntityHint
from format import apply_placeholders


## 1. Surowy tekst + wstępne hinty (np. z rule-based lub spaCy NER)


In [2]:
raw_text = (
    "Reprezentujemy konsorcjum DataSafe, które zajmuje się anonimizacją dokumentów. "
    "Siedziba firmy znajduje się we Wrocławiu przy ulicy Kościuszki 10. "
    "Dane osobowe takich osób jak Jan Kowalski czy Anna Nowak muszą zostać zanonimizowane. "
    "Mój nr telefonu to 123-456-789. Nazywam się Krawiec i urodziłem się 20-10-2024."
)


## 2. Preprocessing spaCy


In [3]:
pre = SpacyPreprocessor()
pre_result = pre(raw_text)
print(pre_result.meta)


{'model_name': 'pl_core_news_md', 'use_ner_hints': True, 'num_tokens': 57, 'num_sentences': 5, 'num_entities': 5}


In [4]:
pre_result.entities

[EntityHint(text='Wrocławiu', label='placeName', start_char=110, end_char=119),
 EntityHint(text='ulicy', label='geogName', start_char=125, end_char=130),
 EntityHint(text='Jan Kowalski', label='persName', start_char=175, end_char=187),
 EntityHint(text='Anna Nowak', label='persName', start_char=192, end_char=202),
 EntityHint(text='Krawiec', label='persName', start_char=276, end_char=283)]

## 3. Chunk per hint (minimalny kontekst)
`sentence_radius=0` → tylko zdanie z hintem. Możesz zwiększyć do 1, aby dodać sąsiadujące zdania.


In [5]:
cm = ContextManager()
chunks = cm.chunk_by_hints(pre_result, hints=pre_result.entities, sentence_radius=0)

for i, ch in enumerate(chunks):
    print(f"Chunk {i}: chars {ch.start_char}-{ch.end_char}, sentences={len(ch.sentences)}, entities={len(ch.entities)}")
    print(ch.text)
    print('---')


Chunk 0: chars 79-145, sentences=1, entities=1
Siedziba firmy znajduje się we Wrocławiu przy ulicy Kościuszki 10.
---
Chunk 1: chars 79-145, sentences=1, entities=1
Siedziba firmy znajduje się we Wrocławiu przy ulicy Kościuszki 10.
---
Chunk 2: chars 146-231, sentences=1, entities=1
Dane osobowe takich osób jak Jan Kowalski czy Anna Nowak muszą zostać zanonimizowane.
---
Chunk 3: chars 146-231, sentences=1, entities=1
Dane osobowe takich osób jak Jan Kowalski czy Anna Nowak muszą zostać zanonimizowane.
---
Chunk 4: chars 264-311, sentences=1, entities=1
Nazywam się Krawiec i urodziłem się 20-10-2024.
---


## 4. Klasyfikacja w Ollama (JSON only)
Model zwraca etykietę z listy albo `none` (odrzuca hint).


In [6]:
classifier = OllamaEntityClassifier()

try:
    classified = classifier.classify_document(chunks)
except Exception as exc:
    print("Nie udało się wywołać Ollama (uruchom serwer na localhost:11434):", exc)
else:
    for (start, end), label in sorted(classified.items()):
        span = raw_text[start:end]
        status = 'OK' if label != 'none' else 'ODRZUCONE'
        print(f"{status:10} {label:18} -> {span!r}")



Tekst dokumentu (fragment):

"""Siedziba firmy znajduje się we Wrocławiu przy ulicy Kościuszki 10."""

Lista kandydatów do klasyfikacji (id, fragment, wstępna etykieta, pozycja w TEKŚCIE FRAGMENTU):
[
  {
    "id": 0,
    "text": "Wrocławiu",
    "hint_label": "city",
    "start": 31,
    "end": 40
  }
]

Dla KAŻDEGO kandydata wybierz JEDNĄ etykietę z listy (lub "none" jeśli nie pasuje):

- name
- surname
- age
- date-of-birth
- date
- sex
- religion
- political-view
- ethnicity
- sexual-orientation
- health
- relative
- city
- address
- email
- phone
- pesel
- document-number
- company
- school-name
- job-title
- bank-account
- credit-card-number
- username
- secret
- none

Zwróć WYŁĄCZNIE JSON w formacie:

{
  "entities": [
    {"id": <liczba>, "label": "<etykieta>"},
    ...
  ]
}

Nie udało się wywołać Ollama (uruchom serwer na localhost:11434): Ollama chat failed (host=http://localhost:11434, model=SpeakLeash/bielik-7b-instruct-v0.1-gguf:Q4_K_S)


## 5. (opcjonalnie) Zamiana na placeholdery
Przydaje się do anonimizacji tekstu na podstawie zatwierdzonych etykiet.


In [ ]:
if 'classified' in locals():
    redacted = apply_placeholders(raw_text, classified)
    print(redacted)
